### Center of Neuroengineering and Therapeutics
# IEEG Pre-processing Pipeline
This notebook provides a recommended usage pipeline for the iEEG pre-processing toolkit.

## Imports



In [8]:
import pandas as pd
import numpy as np
import tools
import os,sys
from ieeg.auth import Session

path = os.getcwd()
sys.path.append(path)

## Pipeline implementation

In [9]:
#@title Login Configuration
#@markdown Set up login account for obtaining data
with open(os.path.join(path,"config.json"), 'rb') as f:
    config = pd.read_json(f, typ='series')

In [12]:
#@title Get Data via Input
#@markdown Specify the filename, start/stop time, electrodes, etc.
iEEG_filename = "HUP172_phaseII"#@param {type:"string"}
start_time = 402580 #@param {type:"number"}
stop_time = 402600 #@param {type:"number"}
electrodes = "LE10, LE11, LH01, LH02, LH03, LH04" #@param {type:"string"}
electrodes = electrodes.split(', ')

In [ ]:
#@title Get Data via File
#@markdown Specify the path to file containing data info (.csv/.tsv)
#@markdown Format Filename | Start time | Stop time | Electrodes
pd.read_csv

In [ ]:
#@title Import data
data, fs = tools.get_iEEG_data(config.usr, config.pwd, iEEG_filename, start_time, stop_time)

# %%
print(data.columns)


Index(['EKG1', 'EKG2', 'LB01', 'LB02', 'LB03', 'LB04', 'LB05', 'LB06', 'LB07',
       'LB08',
       ...
       'RE01', 'RE02', 'RE03', 'RE04', 'RE05', 'RE06', 'RE07', 'RE08', 'RE09',
       'RE10'],
      dtype='object', length=136)


In [ ]:
#@title Clean channel labels
clean_channels = tools.clean_labels(data.columns)

# show cleaned and original labels
if np.all(clean_channels==data.columns):
    print('No channels changed.')
else:
    comp_chan = np.vstack([data.columns,np.array(clean_channels)]).T
    print(comp_chan[np.where(clean_channels!=data.columns)[0],:])

[['EKG1' 'EKG01']
 ['EKG2' 'EKG02']]


In [ ]:
#@title Non-ieeg channels
#Find and return a boolean mask for non ieeg channels
non_ieeg_channels = tools.find_non_ieeg(clean_channels)
print(np.array(clean_channels)[non_ieeg_channels])

['EKG01' 'EKG02']


In [ ]:
#@title Clean data
data.columns = clean_channels
data = data.iloc[:, ~non_ieeg_channels]

#%%
print(data.columns)

Index(['LB01', 'LB02', 'LB03', 'LB04', 'LB05', 'LB06', 'LB07', 'LB08', 'LB09',
       'LB10',
       ...
       'RE01', 'RE02', 'RE03', 'RE04', 'RE05', 'RE06', 'RE07', 'RE08', 'RE09',
       'RE10'],
      dtype='object', length=134)


In [ ]:
#@title Plot data
t_sec = np.linspace(start_time, stop_time, num=data.shape[0])
fig, ax = tools.plot_iEEG_data(data, t_sec)
fig.set_size_inches(18.5, 10.5)
ax.set_title(iEEG_filename)
fig.show()
